# Just change the path varibles to what work for you
# this file apply the Pipline to all dataset

In [1]:
import os
import numpy as np
import SimpleITK as sitk
from nipype.interfaces import fsl
from datetime import datetime
from skimage.io import imread,imsave
import pandas as pd
import cv2 as cv
import cv2
from scipy.stats import entropy
from dltk.io.preprocessing import whitening

220521-13:27:00,576 nipype.utils WARNING:
	 A newer version (1.8.1) of nipy/nipype is available. You are using 1.7.1


/home/forge/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
ADNI_DIR = '/media/forge/Elements/Yacine/Data/ADNI'
REG_DIR = '/media/forge/Elements/Yacine/Data/AFFINE_REGISTRATION'
MAT_DIR = '/media/forge/Elements/Yacine/Data/MAT'
SKS_MASK_DIR = '/media/forge/Elements/Yacine/Data/SKULL_STRIPPING_MASK'
SKS_FSL_DIR = '/media/forge/Elements/Yacine/Data/SKULL_STRIPPING_FSL'
AFFINE_REG_ONLY_2D_IMAGES_DIR = '/media/forge/Elements/Yacine/Data/AFFINE_REG_ONLY_2D'
AFFINE_REG_SKS_MASK_2D_IMAGES_DIR = '/media/forge/Elements/Yacine/Data/AFFINE_REG_SKS_MASK_2D'
AFFINE_REG_SKS_FSL_2D_IMAGES_DIR = '/media/forge/Elements/Yacine/Data/AFFINE_REG_SKS_FSL_2D'
MNI152_PATH = '/home/forge/Desktop/PFE/PFE_Code/res/atlas/MNI_152/MNI152lin_T1_2mm.nii.gz'
MNI152_MASK_PATH = '/home/forge/Desktop/PFE/PFE_Code/res/atlas/MNI_152/MNI152lin_T1_2mm_brain_mask.nii.gz'

In [3]:
if not os.path.isdir(REG_DIR):
    os.makedirs(REG_DIR)

if not os.path.isdir(MAT_DIR):
    os.makedirs(MAT_DIR)

if not os.path.isdir(SKS_MASK_DIR):
    os.makedirs(SKS_MASK_DIR)

if not os.path.isdir(SKS_FSL_DIR):
    os.makedirs(SKS_FSL_DIR)

if not os.path.isdir(AFFINE_REG_ONLY_2D_IMAGES_DIR):
    os.makedirs(AFFINE_REG_ONLY_2D_IMAGES_DIR)

if not os.path.isdir(AFFINE_REG_SKS_MASK_2D_IMAGES_DIR):
    os.makedirs(AFFINE_REG_SKS_MASK_2D_IMAGES_DIR)

if not os.path.isdir(AFFINE_REG_SKS_FSL_2D_IMAGES_DIR):
    os.makedirs(AFFINE_REG_SKS_FSL_2D_IMAGES_DIR)

AFFINE REGISTRATION

In [ ]:
flt = fsl.FLIRT()    
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.reference = MNI152_PATH
flt.inputs.dof = 12
size = len(os.listdir(ADNI_DIR))


for index,file in enumerate(os.listdir(ADNI_DIR)):

    # init path varibles
    reg_input_file_path = f'{ADNI_DIR}/{file}'
    reg_output_file_path = f'{REG_DIR}/{file}'
    mat_out_file_path = f'{MAT_DIR}/{file.replace(".nii","")}.mat' 

    start=datetime.now()
    flt.inputs.in_file = reg_input_file_path
    flt.inputs.out_file = reg_output_file_path
    flt.inputs.out_matrix_file = mat_out_file_path
    res = flt.run()
    end = datetime.now()-start
    
    print(f'{index+1} out of {size} took {end}')


SKULL STRIPPING WITH MASK

In [8]:
size = len(os.listdir(REG_DIR))

mask = fsl.ApplyMask()
mask.inputs.output_type = "NIFTI_GZ"
mask.inputs.mask_file = MNI152_MASK_PATH



for index,file in enumerate(os.listdir(REG_DIR)):

    # init path varibles
    sk_input_file_path = f'{REG_DIR}/{file}'
    sk_output_file_path = f'{SKS_MASK_DIR}/{file}'
   

    # start = datetime.now()
    mask.inputs.in_file = sk_input_file_path
    mask.inputs.out_file = sk_output_file_path
    mask.run()
    # end = datetime.now()-start

    # print(f'{index+1} out of {size} took {end}')
    

SKULL STRIPPING WITH FSL

In [ ]:
# fix the code 
size = len(os.listdir(REG_DIR))

btr = fsl.BET()
btr.inputs.output_type = "NIFTI_GZ"
btr.robust = True
btr.inputs.frac = 0.2

for index,file in enumerate(os.listdir(REG_DIR)):

    # init path varibles
    sk_input_file_path = f'{REG_DIR}/{file}'
    sk_output_file_path = f'{SKS_FSL_DIR}/{file}'
   

    start = datetime.now()
    btr.inputs.in_file = sk_input_file_path
    btr.inputs.out_file = sk_output_file_path
    
    btr.run()
    end = datetime.now()-start

    print(f'{index+1} out of {size} took {end}')

CREATING THE GRID IMAGES

In [25]:
padd = 5
start = 25
end = 70
nb_img = 16
for indx , file in enumerate(os.listdir(SKS_MASK_DIR)):
    array = sitk.GetArrayFromImage(sitk.ReadImage(SKS_MASK_DIR+'/'+file))
    array = np.interp(array, (array.min(), array.max()), (0, 255))

    graid_image = np.array([])
    data = np.array([])
    entpy_data = {}

    for i in range(start,end):
        value,counts = np.unique(array[i,:,:], return_counts=True)
        entpy_data[i] = entropy(counts, base=2)
    entpy_data = {k: v for k, v in sorted(entpy_data.items(),reverse=True, key=lambda item: item[1])}
    index_of_slices = list(entpy_data.keys())[0:nb_img]

    
    for i , max_indx in enumerate(index_of_slices):
        if (i+1) % 4 == 0:
            data = np.hstack((data,array[max_indx,:,:]))
            if graid_image.size < 1:
                graid_image = data.copy()
            else:
                graid_image = np.vstack((graid_image,data))
            data = np.array([])
              
        else:
            if data.size < 1:
                data = array[max_indx,:,:]
            else:
                data = np.hstack((data,array[max_indx,:,:]))
    imsave(f'{AFFINE_REG_SKS_MASK_2D_IMAGES_DIR}/{file.replace(".nii.gz","")}.png',cv.equalizeHist(np.uint8(graid_image)))